# A very quick data scrape...

#### Note to self - try this with Beautiful Soup Later

So here I'm just importing the packages I need and using pandas to read in the table

In [25]:
import lxml
import pandas as pd

In [55]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [60]:
df.columns =  ['PostalCode','Borough','Neighborhood']
pd.set_option("display.max_rows", None, "display.max_columns", None)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


And then there are those funky not assigned values. I read through the database, and there are no places where we have a not assigned without a NaN and vice versa, so I don't need to do anything fancy to that. I can just drop the not assigneds. And now there are no NaN's either.

In [63]:
# Get names of indexes for which column Age has value 30
indexNames = df[ df['Borough'] == "Not assigned" ].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.reset_index(inplace=True)
df.drop(['index'], axis=1,inplace = True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Now again, wikipedia has changed their table. I don't have to combine rows (although it would be really useful to know how to do that), because they've already done it for us and separated the neighborhoods with slashes. So I just need to replace the slashes with commas. And use df.shape to find out how many rows I need to iterate over.

In [64]:
df.shape

(103, 3)

In [53]:
for x in range(0,103):
    df.loc[x,"Neighborhood"] = df.loc[x,"Neighborhood"].replace(' /',',')
    
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [54]:
df.shape

(103, 3)

And there you have it. All completed. Nothing too special about this one.